# Convolutional Neural Network

### Importing the libraries

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from glob import glob
from sklearn.model_selection import train_test_split
import os

In [0]:
tf.__version__

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [0]:
# train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True)
# training_set = train_datagen.flow_from_directory('dataset/training_set',
#                                                  target_size = (64, 64),
#                                                  batch_size = 32,
#                                                  class_mode = 'binary')

### Preprocessing the Test set

In [0]:
# test_datagen = ImageDataGenerator(rescale = 1./255)
# test_set = test_datagen.flow_from_directory('dataset/test_set',
#                                             target_size = (64, 64),
#                                             batch_size = 32,
#                                             class_mode = 'binary')

In [10]:
df= pd.read_csv(r'C:\Users\hp\Desktop\coding\ML projects\Audio_Classifications\Multimodular\Cat&Dog\image\dataset\cat_dog.csv')
df

,image,labels
0,dog.11289.jpg,1
1,cat.10836.jpg,0
2,cat.7970.jpg,0
3,cat.2212.jpg,0
4,dog.2653.jpg,1
...,...,...
24995,dog.12201.jpg,1
24996,dog.2893.jpg,1
24997,cat.12267.jpg,0
24998,dog.4380.jpg,1


In [6]:
base_adress=r'C:\Users\hp\Desktop\coding\ML projects\Audio_Classifications\Multimodular\Cat&Dog\image\dataset\cat_dog'


In [37]:
df['image_path'] = df['image'].apply(lambda x: os.path.join(base_adress, x))

# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['labels'], random_state=42)

# Function to load and preprocess image
def load_image(image_path, label, img_size=(64, 64)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # or decode_png
    image = tf.image.resize(image, img_size)
    image = image / 255.0  # normalize to [0,1]
    return image, label

# Convert to TensorFlow Datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_df['image_path'].values, train_df['labels'].values))
train_ds = train_ds.map(load_image).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_df['image_path'].values, test_df['labels'].values))
test_ds = test_ds.map(load_image).batch(32).prefetch(tf.data.AUTOTUNE)


In [38]:
train_ds.element_spec

(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

## Part 2 - Building the CNN

### Initialising the CNN

In [39]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [40]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [41]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [42]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [43]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [44]:
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [45]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [46]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [47]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [48]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [49]:
cnn.fit(x = train_ds, validation_data = test_ds, epochs = 5)

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 30s 45ms/step - accuracy: 0.5771 - loss: 0.6673 - val_accuracy: 0.7250 - val_loss: 0.5529
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.7321 - loss: 0.5346 - val_accuracy: 0.7678 - val_loss: 0.4843
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.7856 - loss: 0.4539 - val_accuracy: 0.8038 - val_loss: 0.4281
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.8223 - loss: 0.3925 - val_accuracy: 0.8126 - val_loss: 0.4078
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - accuracy: 0.8444 - loss: 0.3483 - val_accuracy: 0.8220 - val_loss: 0.4150


## Part 4 - Making a single prediction

In [50]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img(r'C:\Users\hp\Desktop\coding\ML projects\Audio_Classifications\Multimodular\Cat&Dog\image\dataset\cat_dog\cat.2000.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)

if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


In [51]:
print(prediction)
result

dog


array([[1.]], dtype=float32)

In [52]:
result

array([[1.]], dtype=float32)